In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 5.2 MB/s 


In [ ]:
import pandas as pd
import spacy
import faiss


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv('medium_Data.csv')
df

,Titles,Authors,Dates,Links,Contents
0,Please Consider the Racial Impact of Your Hall...,Savala Nolan,Oct 17,https://momentum.medium.com/i-beg-you-consider...,"Oh, it’s that time of year again! So here is a..."
1,That Time We Burned Down Players’ Houses in Ul...,Tim Cotten,Oct 3,https://blog.cotten.io/that-time-we-burned-dow...,Ultima Online is celebrating its 25th annivers...
2,The Wonderful Outcome of the Ice Bucket Challenge,Jamie Cohen,Sep 30,https://newanddigital.medium.com/the-long-payo...,The water had to be ice cold and you had to be...
3,Why Should You Care About the Bar Exam?,Dan Canon,Sep 27,https://medium.com/i-taught-the-law/why-should...,You didn’t take the bar exam. That’s probably ...
4,Please Don’t Tell My Daughter She’s Beautiful,Kerala Taylor,Oct 11,https://keralataylor.medium.com/please-dont-te...,I have two beautiful children.I might be biase...
...,...,...,...,...,...
304,How I Kept My Sense of Humor Through Vision Loss,Nancy Solari,Sep 15,https://betterhumans.pub/how-i-kept-my-sense-o...,My diagnosis of retinitis pigmentosa at age 16...
305,The true scientific meaning of Mercury enterin...,Ethan Siegel,Sep 15,https://medium.com/starts-with-a-bang/the-true...,"Throughout almost all of the year, you can see..."
306,"The Feelings Monster, building a character wit...",Microsoft Design,Sep 6,https://medium.com/microsoft-design/the-feelin...,On designing for mental health and the power o...
307,Feeling Overwhelmed? You Need an MVD—That’s Mi...,Rebecca Pendleton,"May 27, 2021",https://index.medium.com/feeling-overwhelmed-y...,I’\nm a product director at a corporate financ...


In [ ]:
import re
import string

alphanumeric = lambda x: re.sub(r"""\S*\d{1}\w*""", ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
new_line = lambda x: re.sub(r'(?<!\.)\n', '', x)

df['Titles'] = df.Titles.map(alphanumeric).map(punc_lower).map(new_line)
df.head(10)

,Titles,Authors,Dates,Links,Contents
0,please consider the racial impact of your hall...,Savala Nolan,Oct 17,https://momentum.medium.com/i-beg-you-consider...,"Oh, it’s that time of year again! So here is a..."
1,that time we burned down players’ houses in ul...,Tim Cotten,Oct 3,https://blog.cotten.io/that-time-we-burned-dow...,Ultima Online is celebrating its 25th annivers...
2,the wonderful outcome of the ice bucket challenge,Jamie Cohen,Sep 30,https://newanddigital.medium.com/the-long-payo...,The water had to be ice cold and you had to be...
3,why should you care about the bar exam,Dan Canon,Sep 27,https://medium.com/i-taught-the-law/why-should...,You didn’t take the bar exam. That’s probably ...
4,please don’t tell my daughter she’s beautiful,Kerala Taylor,Oct 11,https://keralataylor.medium.com/please-dont-te...,I have two beautiful children.I might be biase...
5,how google search is reinventing itself in,Eric Chung,Oct 7,https://uxdesign.cc/how-google-search-is-reinv...,Google is completely changing the way that sea...
6,chatgpt is having a thomas edison moment,Thomas Smith,Dec 9,https://tomsmith585.medium.com/chatgpt-is-havi...,ChatGPT is blowing up. Twitter is inundated wi...
7,how to be a tourist,Roblin Meeks,Sep 26,https://humanparts.medium.com/how-to-be-a-tour...,Paris in the summer is hot and not really buil...
8,theses on the concept of nonhuman intelligence,Blaise Aguera y Arcas,Sep 19,https://medium.com/@blaisea/1011-theses-on-the...,When Gottfried Wilhelm Leibniz discovered bina...
9,a ’s guide to mastodon,Jeremy Littau,Nov 8,https://jeremylittau.medium.com/a-n00bs-guide-...,From “The Axe Forgets” — S1E5 of Andor on Disn...


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english'))
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
df['Titles'] = df['Titles'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
mask =df.isnull()
mask_ = np.sum(mask)
mask_

Titles       0
Authors      0
Dates        0
Links        0
Contents    11
dtype: int64

In [ ]:
df = df.dropna(subset=["Contents"])

In [ ]:
mask =df.isnull()
mask_ = np.sum(mask)
mask_

Titles      0
Authors     0
Dates       0
Links       0
Contents    0
dtype: int64

In [ ]:
df.to_csv("data.csv")


In [ ]:
import faiss
import nltk
nltk.download('punkt')
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data = []
with open("data.csv", "r") as file:
    for line in file:
        data.append(line.strip())

In [ ]:
from transformers import BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

from transformers import AutoModel
bert_model = AutoModel.from_pretrained("bert-base-uncased")


input_data = " ".join(df["Titles"].tolist())

tokens = bert_tokenizer.tokenize(input_data)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokens = [token for token in tokens if isinstance(token, str)]

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 54.1 MB/s 
     |████████████████████████████████| 182 kB 32.7 MB/s 


In [ ]:
from transformers import BertModel
import torch

model = BertModel.from_pretrained("bert-base-uncased")
embeddings = []
for tokens in tokens:
    # Convert the tokens list to a one-dimensional array or list
    tokens_1d = [token for token in tokens if token not in stop_words]
    print(tokens_1d)

    # Convert the tokens array or list to a tensor
    tokens_tensor = torch.tensor(tokens_1d)
    # Create an embedding for the tensor using the BERT model
    embedding = model(tokens_tensor)[0][0]s
    embeddings.append(embedding)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[',', 'T', 'l', 'e', ',', 'A', 'u', 'h', 'r', ',', 'D', 'e', ',', 'L', 'n', 'k', ',', 'C', 'n', 'e', 'n']


ValueError: ignored

In [ ]:
# nlp = spacy.load("en_core_web_sm")
# vectors = [nlp(text).vector for text in df["Titles"]]
# vectors = np.array(vectors)
# vectors = np.atleast_2d(vectors)
# mask = np.isnan(vectors)
# index = faiss.IndexFlatL2(vectors[0].shape[0])
# index
# index.add(vectors)